In [7]:
import machine
import numpy as np
import math

# the distance matrix to be used in the
# traveling salesman problem
distances = np.matrix([
    [0,10,20,5,18],
    [0,0,15,32,10],
    [0,0,0,25,16],
    [0,0,0,0,35],
    [0,0,0,0,0]
])
distances = distances + distances.T

In [9]:
b = machine.boltzmann(hamiltonian_error_charge=0.5, bias_charge=-0.2)
b.create_network(distances)
machine.anneal(b,T=5000,schedule=lambda T: math.log10(T) if T > 10 else 0.01)

-- annealing <machine.boltzmann object at 0x10cc4e630> --
iterating, current T=2000...
C->B->E->A->D->C
distance=73
iterating, current T=1996.698970004336...
C->B->E->A->D->C
distance=73
iterating, current T=1993.398657410432...
C->B->E->A->D->C
distance=73
iterating, current T=1990.099063249012...
C->B->E->A->D->C
distance=73
iterating, current T=1986.8001885537321...
C->B->E->A->D->C
distance=73
iterating, current T=1983.502034361193...
C->B->E->A->D->C
distance=73
iterating, current T=1980.2046017109524...
C->B->E->A->D->C
distance=73
iterating, current T=1976.9078916455376...
C->B->E->A->D->C
distance=73
iterating, current T=1973.6119052104586...
C->B->E->A->D->C
distance=73
iterating, current T=1970.3166434542204...
C->B->E->A->D->C
distance=73
iterating, current T=1967.0221074283359...
C->B->E->A->D->C
distance=73
iterating, current T=1963.7282981873386...
C->A->E->B->D->C
distance=105
iterating, current T=1960.4352167887964...
C->A->E->B->D->C
distance=105
iterating, current T=1

C->A->E->B->D->C
distance=105
iterating, current T=1609.3148702391388...
C->A->E->B->D->C
distance=105
iterating, current T=1606.10822921491...
C->A->E->B->D->C
distance=105
iterating, current T=1602.9024544076117...
C->A->E->B->D->C
distance=105
iterating, current T=1599.6975473137052...
C->A->E->B->D->C
distance=105
iterating, current T=1596.4935094347675...
C->A->E->B->D->C
distance=105
iterating, current T=1593.2903422775182...
C->A->E->B->D->C
distance=105
iterating, current T=1590.0880473538455...
C->A->E->B->D->C
distance=105
iterating, current T=1586.8866261808325...
C->A->E->B->D->C
distance=105
iterating, current T=1583.686080280784...
C->A->E->B->D->C
distance=105
iterating, current T=1580.4864111812533...
C->A->E->B->D->C
distance=105
iterating, current T=1577.2876204150698...
C->A->E->B->D->C
distance=105
iterating, current T=1574.0897095203652...
C->A->E->B->D->C
distance=105
iterating, current T=1570.8926800406018...
C->A->E->B->D->C
distance=105
iterating, current T=156

B->A->E->C->D->B
distance=101
iterating, current T=1215.7301106516938...
B->A->E->C->D->B
distance=101
iterating, current T=1212.6452734784546...
B->A->E->C->D->B
distance=101
iterating, current T=1209.5615397000952...
B->A->E->C->D->B
distance=101
iterating, current T=1206.478911730934...
B->A->E->C->D->B
distance=101
iterating, current T=1203.3973919957368...
B->A->E->C->D->B
distance=101
iterating, current T=1200.3169829297838...
B->A->E->C->D->B
distance=101
iterating, current T=1197.2376869789375...
B->A->E->C->D->B
distance=101
iterating, current T=1194.1595065997124...
B->A->E->C->D->B
distance=101
iterating, current T=1191.0824442593432...
B->A->E->C->D->B
distance=101
iterating, current T=1188.0065024358557...
B->A->E->C->D->B
distance=101
iterating, current T=1184.9316836181363...
B->A->E->C->D->B
distance=101
iterating, current T=1181.8579903060036...
B->A->E->C->D->B
distance=101
iterating, current T=1178.7854250102803...
B->A->E->C->D->B
distance=101
iterating, current T=1

C->A->E->B->D->C
distance=105
iterating, current T=836.6402959787769...
C->A->E->B->D->C
distance=105
iterating, current T=833.7177572006506...
C->A->E->B->D->C
distance=105
iterating, current T=830.796738149097...
C->A->E->B->D->C
distance=105
iterating, current T=827.8772433663454...
C->A->E->B->D->C
distance=105
iterating, current T=824.9592774214458...
C->A->E->B->D->C
distance=105
iterating, current T=822.0428449105051...
C->A->E->B->D->C
distance=105
iterating, current T=819.127950456927...
C->A->E->B->D->C
distance=105
iterating, current T=816.2145987116535...
C->A->E->B->D->C
distance=105
iterating, current T=813.3027943534103...
C->A->E->B->D->C
distance=105
iterating, current T=810.3925420889544...
C->A->E->B->D->C
distance=105
iterating, current T=807.4838466533255...
C->A->E->B->D->C
distance=105
iterating, current T=804.5767128100988...
C->A->E->B->D->C
distance=105
iterating, current T=801.6711453516428...
C->A->E->B->D->C
distance=105
iterating, current T=798.76714909937

D->A->E->B->C->D
distance=73
iterating, current T=482.19132237608846...
D->A->E->B->C->D
distance=73
iterating, current T=479.5081029856383...
D->A->E->B->C->D
distance=73
iterating, current T=476.827307035128...
D->A->E->B->C->D
distance=73
iterating, current T=474.14894591642627...
D->A->E->B->C->D
distance=73
iterating, current T=471.4730311270128...
D->A->E->B->C->D
distance=73
iterating, current T=468.7995742714404...
D->A->E->B->C->D
distance=73
iterating, current T=466.12858706282384...
D->A->E->B->C->D
distance=73
iterating, current T=463.4600813243561...
D->A->E->B->C->D
distance=73
iterating, current T=460.79406899085325...
D->A->E->B->C->D
distance=73
iterating, current T=458.1305621103277...
D->A->E->B->C->D
distance=73
iterating, current T=455.46957284559124...
D->A->E->B->C->D
distance=73
iterating, current T=452.8111134758885...
D->A->E->B->C->D
distance=73
iterating, current T=450.1551963985607...
D->A->E->B->C->D
distance=73
iterating, current T=447.50183413074194...
D

...

B->E->D->A->C->B
distance=85
iterating, current T=7.481924356441233...
B->E->D->A->C->B
distance=85
iterating, current T=7.471924356441233...
B->E->D->A->C->B
distance=85
iterating, current T=7.461924356441234...
B->E->D->A->C->B
distance=85
iterating, current T=7.451924356441234...
B->E->D->A->C->B
distance=85
iterating, current T=7.441924356441234...
B->E->D->A->C->B
distance=85
iterating, current T=7.431924356441234...
B->E->D->A->C->B
distance=85
iterating, current T=7.4219243564412345...
B->E->D->A->C->B
distance=85
iterating, current T=7.411924356441235...
B->E->D->A->C->B
distance=85
iterating, current T=7.401924356441235...
B->E->D->A->C->B
distance=85
iterating, current T=7.391924356441235...
B->E->D->A->C->B
distance=85
iterating, current T=7.381924356441235...
B->E->D->A->C->B
distance=85
iterating, current T=7.371924356441236...
B->E->D->A->C->B
distance=85
iterating, current T=7.361924356441236...
B->E->D->A->C->B
distance=85
iterating, current T=7.351924356441236...
B->E-

B->E->D->A->C->B
distance=85
iterating, current T=6.191924356441261...
B->E->D->A->C->B
distance=85
iterating, current T=6.181924356441261...
B->E->D->A->C->B
distance=85
iterating, current T=6.171924356441261...
B->E->D->A->C->B
distance=85
iterating, current T=6.161924356441261...
B->E->D->A->C->B
distance=85
iterating, current T=6.151924356441262...
B->E->D->A->C->B
distance=85
iterating, current T=6.141924356441262...
B->E->D->A->C->B
distance=85
iterating, current T=6.131924356441262...
B->E->D->A->C->B
distance=85
iterating, current T=6.121924356441262...
B->E->D->A->C->B
distance=85
iterating, current T=6.111924356441262...
B->E->D->A->C->B
distance=85
iterating, current T=6.101924356441263...
B->E->D->A->C->B
distance=85
iterating, current T=6.091924356441263...
B->E->D->A->C->B
distance=85
iterating, current T=6.081924356441263...
B->E->D->A->C->B
distance=85
iterating, current T=6.071924356441263...
B->E->D->A->C->B
distance=85
iterating, current T=6.0619243564412635...
B->E-

distance=85
iterating, current T=4.941924356441287...
B->E->D->A->C->B
distance=85
iterating, current T=4.931924356441288...
B->E->D->A->C->B
distance=85
iterating, current T=4.921924356441288...
B->E->D->A->C->B
distance=85
iterating, current T=4.911924356441288...
B->E->D->A->C->B
distance=85
iterating, current T=4.901924356441288...
B->E->D->A->C->B
distance=85
iterating, current T=4.891924356441288...
B->E->D->A->C->B
distance=85
iterating, current T=4.881924356441289...
B->E->D->A->C->B
distance=85
iterating, current T=4.871924356441289...
B->E->D->A->C->B
distance=85
iterating, current T=4.861924356441289...
B->E->D->A->C->B
distance=85
iterating, current T=4.851924356441289...
B->E->D->A->C->B
distance=85
iterating, current T=4.8419243564412895...
B->E->D->A->C->B
distance=85
iterating, current T=4.83192435644129...
B->E->D->A->C->B
distance=85
iterating, current T=4.82192435644129...
B->E->D->A->C->B
distance=85
iterating, current T=4.81192435644129...
B->E->D->A->C->B
distance

B->E->D->C->A->B
distance=100
iterating, current T=3.6619243564413146...
B->E->D->C->A->B
distance=100
iterating, current T=3.651924356441315...
B->E->D->C->A->B
distance=100
iterating, current T=3.641924356441315...
B->E->D->C->A->B
distance=100
iterating, current T=3.6319243564413153...
B->E->D->C->A->B
distance=100
iterating, current T=3.6219243564413155...
B->E->D->C->A->B
distance=100
iterating, current T=3.6119243564413157...
B->E->D->C->A->B
distance=100
iterating, current T=3.601924356441316...
B->E->D->C->A->B
distance=100
iterating, current T=3.591924356441316...
B->E->D->C->A->B
distance=100
iterating, current T=3.5819243564413163...
B->E->D->C->A->B
distance=100
iterating, current T=3.5719243564413166...
B->E->D->C->A->B
distance=100
iterating, current T=3.5619243564413168...
B->E->D->C->A->B
distance=100
iterating, current T=3.551924356441317...
B->E->D->C->A->B
distance=100
iterating, current T=3.541924356441317...
B->E->D->C->A->B
distance=100
iterating, current T=3.5319

B->A->D->C->E->B
distance=66
iterating, current T=2.1219243564413475...
B->A->D->C->E->B
distance=66
iterating, current T=2.1119243564413477...
B->A->D->C->E->B
distance=66
iterating, current T=2.101924356441348...
B->A->D->C->E->B
distance=66
iterating, current T=2.091924356441348...
B->A->D->C->E->B
distance=66
iterating, current T=2.0819243564413483...
B->A->D->C->E->B
distance=66
iterating, current T=2.0719243564413485...
B->A->D->C->E->B
distance=66
iterating, current T=2.0619243564413487...
B->A->D->C->E->B
distance=66
iterating, current T=2.051924356441349...
B->A->D->C->E->B
distance=66
iterating, current T=2.041924356441349...
B->A->D->C->E->B
distance=66
iterating, current T=2.0319243564413494...
B->A->D->C->E->B
distance=66
iterating, current T=2.0219243564413496...
B->A->D->C->E->B
distance=66
iterating, current T=2.01192435644135...
B->A->D->C->E->B
distance=66
iterating, current T=2.00192435644135...
B->A->D->C->E->B
distance=66
iterating, current T=1.99192435644135...
B-

In [3]:
distances

matrix([[ 0, 10, 20,  5, 18],
        [10,  0, 15, 32, 10],
        [20, 15,  0, 25, 16],
        [ 5, 32, 25,  0, 35],
        [18, 10, 16, 35,  0]])

In [108]:
b = machine.boltzmann(hamiltonian_error_charge=0.1, bias_charge=-0.2)
b.create_network(distances)

In [89]:
cities,epoch = b.get_states().shape

In [90]:
cities_left = [ i for i in range(cities) ]

In [91]:
cities

5

In [4]:
import random

In [5]:
sequence = []
for i in range(cities):
    selected = random.randint(0,len(cities_left)-1)
    print(selected)
    print(cities_left)
    sequence.append([cities_left[selected],i])
    del cities_left[selected]
sequence.append([sequence[0][0],cities])

NameError: name 'cities' is not defined

In [13]:
m = b.get_states()

In [21]:
for i,c in enumerate(m[:,0].A1): print(i)

0
1
2
3
4


In [ ]:
get_active_index = lambda epoch: for i,c in enumerate(m[:,0].A1)

In [25]:
arr = [0,0,1]

In [26]:
arr.index(1)

2

In [27]:
t1 = 6
t2 = 0

In [29]:
t2 % 6

0

In [31]:
abs(t2 - 6)

6

In [33]:
m.A1

array([0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0])